In [18]:
df = spark.sql("SELECT * FROM News_Lake_db.dbo.tbl_latest_news")
display(df)

StatementMeta(, 31ea7c75-52cb-44ce-82a0-8358ad054704, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 527c5105-a4b3-453b-8b74-01f0173f6895)

In [19]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 31ea7c75-52cb-44ce-82a0-8358ad054704, 21, Finished, Available, Finished)

In [20]:
# Import the model and configure the input and output columns

model = (AnalyzeText ()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))

StatementMeta(, 31ea7c75-52cb-44ce-82a0-8358ad054704, 22, Finished, Available, Finished)

In [21]:
#Apply model to our dataframe

result = model.transform(df)

StatementMeta(, 31ea7c75-52cb-44ce-82a0-8358ad054704, 23, Finished, Available, Finished)

In [22]:
display(result)

StatementMeta(, 31ea7c75-52cb-44ce-82a0-8358ad054704, 24, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 827d49ba-8547-49d9-b7eb-068a4b632117)

In [23]:
# Create a Sentiment column

from pyspark.sql.functions import col
sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))

StatementMeta(, 31ea7c75-52cb-44ce-82a0-8358ad054704, 25, Finished, Available, Finished)

In [24]:
display(sentiment_df)

StatementMeta(, 31ea7c75-52cb-44ce-82a0-8358ad054704, 26, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 89a7cfcc-085e-417e-bf80-36c826a83cad)

In [25]:
sentiment_df_final = sentiment_df.drop("error","response")

StatementMeta(, 31ea7c75-52cb-44ce-82a0-8358ad054704, 27, Finished, Available, Finished)

In [26]:
display(sentiment_df_final)

StatementMeta(, 31ea7c75-52cb-44ce-82a0-8358ad054704, 28, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3d0560f0-5d4b-412e-9a91-b061fb0efff0)

In [27]:
from pyspark.sql.functions import col, to_date

sentiment_df_final = sentiment_df_final.withColumn("datePublished", to_date(col("datePublished"), "dd-MMM-yyyy"))

StatementMeta(, 31ea7c75-52cb-44ce-82a0-8358ad054704, 29, Finished, Available, Finished)

#### **Type 1 Merge**

In [28]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'News_Lake_db.dbo.tbl_sentiment_analysis'
    sentiment_df_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table Already Exists")

    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")

    spark.sql(f"""  MERGE INTO {table_name} target_table
                    USING vw_sentiment_df_final source_view
                    ON source_view.url = target_table.url
                    WHEN MATCHED AND
                    source_view.title <> target_table.title OR
                    source_view.description <> target_table.description OR
                    source_view.category <> target_table.category OR
                    source_view.image <> target_table.image OR
                    source_view.provider <> target_table.provider OR
                    source_view.datePublished <> target_table.datePublished 
                      
                    THEN UPDATE SET *

                    WHEN NOT MATCHED THEN INSERT *
                    
                """ )


StatementMeta(, 31ea7c75-52cb-44ce-82a0-8358ad054704, 30, Finished, Available, Finished)

Table Already Exists


In [29]:
display(sentiment_df_final)

StatementMeta(, 31ea7c75-52cb-44ce-82a0-8358ad054704, 31, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1f0ade0d-a512-4636-bfb3-17f551d0bcc5)

In [2]:
spark.sql("""
CREATE OR REPLACE TABLE News_Lake_db.dbo.tbl_sentiment_analysis
AS
SELECT * FROM News_Lake_db.dbo.tbl_sentiment_analysis
""")

StatementMeta(, 488522c3-79c6-47b4-a85e-f004cb2d90a9, 4, Finished, Available, Finished)

DataFrame[]